# Imports

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
import time
import openai
from enum import Enum
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Driver

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [5]:
URL = "http://localhost:8080"
driver.get(URL)

# Utils

In [9]:
def get_element_xpath(driver, element):
    xpath_script = """
        function getPathTo(element) {
            // if (element.id !== '')
            //     return 'id(\"'+element.id+'\")';
            if (element === document.body)
                return element.tagName;
            var ix= 0;
            var siblings= element.parentNode.childNodes;
            for (var i= 0; i<siblings.length; i++) {
                var sibling= siblings[i];
                if (sibling===element)
                    return getPathTo(element.parentNode) + '/' + element.tagName + '[' + (ix + 1) + ']';
                if (sibling.nodeType===1 && sibling.tagName===element.tagName)
                    ix++;
            }
        }
        const path = getPathTo(arguments[0]);
        if (path.startsWith('id(')) {
            return path;
        }
        return '//' + path;
    """
    
    xpath = driver.execute_script(xpath_script, element)
    return xpath

In [10]:
driver.get(URL)
driver.find_element(By.XPATH, '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/A[1]').click()
driver.find_element(By.XPATH, '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/UL[1]/LI[1]/A[1]').click()

In [17]:
gpt3_completion_prompt = """You are an agent filling forms on a website. You can issue these commands:
    - FILL////X////"Y": Fill the <input> or <textarea> element that has a relative xpath X with value Y
    - SELECT////X////"Y": Select the option Y in the <select> element that has a relative xpath X
Each command must be issued in a new line, with no preceding or trailing text values.
You are given a form to fill. The form's html is as follows:
{form_html}
The xpath in this format must be relative to the form element. Just give me the commands and nothing else.
YOUR COMMANDS:"""


def fill_form_gpt3(form, zero_shot=True):
    
    for command in commands:
        try:
            print(command)
            cmd, xpath, value = command.split('////')
            complete_xpath = f'{base_xpath}/{xpath}'
            values[complete_xpath] = value
            if cmd == 'FILL':
                form.find_element(By.XPATH, complete_xpath).send_keys(value)
            elif cmd == 'SELECT':
                select = Select(form.find_element(By.XPATH, complete_xpath))
                select.select_by_visible_text(value)
        except:
            continue
    return values

In [19]:
form = driver.find_element(By.TAG_NAME, 'form')
base_xpath = get_element_xpath(driver, form)
form_html = form.get_attribute('outerHTML')
prompt = gpt3_completion_prompt.format(form_html=form_html)
res = openai.Completion.create(
    model='text-davinci-003',
    prompt=prompt,
    temperature=0.5,
    max_tokens=512,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
values = {}
commands = res.choices[0].text.split('\n')[1:]
print(commands)

['FILL////input[@id="lastName"]////"Smith"']


In [21]:
for command in commands:
    try:
        print(command)
        cmd, xpath, value = command.split('////')
        complete_xpath = f'{base_xpath}/{xpath}'
        print(complete_xpath)
        values[complete_xpath] = value
        if cmd == 'FILL':
            form.find_element(By.XPATH, complete_xpath).send_keys(value)
        elif cmd == 'SELECT':
            select = Select(form.find_element(By.XPATH, complete_xpath))
            select.select_by_visible_text(value)
    except:
        continue

FILL////input[@id="lastName"]////"Smith"
//BODY/APP-ROOT[1]/APP-OWNER-LIST[1]/DIV[1]/DIV[1]/FORM[1]/input[@id="lastName"]


In [24]:
form.find_element(By.XPATH, '//input[@id="lastName"]')

<selenium.webdriver.remote.webelement.WebElement (session="0ce58cb724d0e5738eb8ba8d90871965", element="1AA8FF6FBD2CA6356A85E3198AF2BDE3_element_108")>